In [73]:
import pandas as pd
import re
import pymorphy2
import numpy as np
import multiprocessing
from collections import Counter
from keras.preprocessing.sequence import pad_sequences
from joblib import Parallel, delayed
from tqdm import tqdm
from gensim.models.wrappers.fasttext import FastTextKeyedVectors



In [88]:
for i in morph.parse('собака'):
    print(i)

Parse(word='собака', tag=OpencorporaTag('NOUN,anim,femn sing,nomn'), normal_form='собака', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'собака', 403, 0),))


In [90]:
morph = pymorphy2.MorphAnalyzer()
data = pd.read_csv('new_apps_compatible.csv', sep=';').drop('Пустая колонка для совместимости', axis=1)


#Параллельно выполняющийся map функции func по массиву massive
def parallelization(func,massive, tq=True):
    
    num_cores = multiprocessing.cpu_count() # Число наших ядер
    if tq:
        results = np.array(Parallel(n_jobs=num_cores)(delayed(func)(i) for i in tqdm(massive)))
        return results
    else:
        results = Parallel(n_jobs=num_cores)(delayed(func)(i) for i in massive)
        return results

def _word2canonical4w2v(word):
    elems = morph.parse(word)
    my_tag = ''
    res = []
    for elem in elems:
        if 'VERB' in elem.tag or 'GRND' in elem.tag or 'INFN' in elem.tag:
            my_tag = 'V'
        if 'NOUN' in elem.tag:
            my_tag = 'S'
        normalised = elem.normalized.word
        res.append((normalised, my_tag))
    tmp = list(filter(lambda x: x[1] != '', res))
    if len(tmp) > 0:
        return tmp[0]
    else:
        return res[0]

    
def word2canonical(word):
    return _word2canonical4w2v(word)[0]


def getWords(text, filter_short_words=False):
    if filter_short_words:
        return filter(lambda x: len(x) > 3, re.findall(r'(?u)\w+', text))
    else:
        return re.findall(r'(?u)\w+', text)


def text2canonicals(text, add_word=False, filter_short_words=True):
    words = []
    for word in getWords(text, filter_short_words=filter_short_words):
        words.append(word2canonical(word.lower()))
        if add_word:
            words.append(word.lower())
    return words

In [84]:
texts = parallelization(text2canonicals, data.Description.values, True)

100%|██████████| 1720/1720 [00:32<00:00, 52.60it/s]


In [43]:
fasttext.most_similar_cosmul(['женщина', 'король'], '')

[('-король', 0.9802986979484558),
 ('королева', 0.9768819808959961),
 ('короля', 0.9556112289428711),
 ('короли', 0.9534466862678528),
 ('королевства', 0.9524538516998291),
 ('герцогиня', 0.9472317695617676),
 ('королю', 0.9463006854057312),
 ('королевич', 0.9376259446144104),
 ('королевстве', 0.9376195669174194),
 ('королевишна', 0.9349520206451416)]

In [53]:
import tensorflow as tf

In [55]:
import keras.backend as K

Using TensorFlow backend.


In [ ]:
fasttext_model_path = '/Users/egor/Downloads/187/model.model'

In [61]:
class DataProcessor(object):
    def __init__(self):
        pass

In [62]:
class SiameseNetwork(object):
    def __init__(self, embeddings_path=None):
        self.loss = None
        self.model = None
        
    def get_embeddings(self):
        assert self.embeddings_path is not None, 'you must specify embeddings path'
        fasttext = FastTextKeyedVectors.load(self.embeddings_path)
    
    def get_loss(self):
        # Implement pairwise loss
        pass
    
    def get_prob(self, s1, s2):
        return 1 / (1 + tf.exp(s1 - s2))
    
    def build(self):
        # build arch of siamese network
        pass
    
    def train(self):
        pass
    
    def predict(self, X):
        pass